# Predicting Solar Flares

This is a Logistic Regression Machine Learning model, made from the data and research present in the following research paper.

> Song, H., Tan, C., Jing, J. et al. Statistical Assessment of Photospheric Magnetic Features in Imminent Solar Flare Predictions. Sol Phys 254, 101–125 (2009). https://doi.org/10.1007/s11207-008-9288-3

## Introduction

Solar flares are large eruptions of electromagnetic radiation from the Sun lasting from minutes to hours. These solar flares can disrupt communication in space and earth too. 

In this notebook, we try to predict the class of solar flares that could occur based on these 4 factors:

1. Lgnl - The length of the strong-gradient neutral line. it describes the global nonpotentiality of an active region.
1. Tflux - The total unsigned magnetic flux. it is a measure of the active region’s size.
1. Ediss - The total magnetic energy dissipation of Bz.
1. Fidx - flare productivity of a given active region in a day.

where Bz is the line-of-sight components of the magnetic field measured in the plane (x,y).

#### Flare classificaion

In the flare event list, 
- 34 of them (Level = 3) produced X-class flares,
- 68 (Level = 2) produced M-class flares,
- 65 (Level = 1) produced C-class flares.
- the rest of the sample (Level = 0), they either did not produce any flares
or produced smaller flares under C-class in the period.

## The Model

We are using LogisticRegression(sklearn) to train our model to predict the levels of solar flares(0-3) that could occur.

We used Multinomial Logistic Regression which is a modified version of logistic regression that predicts a multinomial probability (i.e. more than two classes which is our case of level 0-3) for each input example.

In [1]:
# import libraries

import pandas as pd
import numpy as np
import sklearn
import sklearn.model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### Description Of Data: 

From 1998 to 2005, a total of 230 flare events are analyzed. The data is sourced from study of *Photospheric Magnetic Features
in Imminent Solar Flare Predictions*. It can be found in the research paper. The dataset contains 230 observations and 5 variables. 

1. Lgnl
1. Tflux
1. Ediss
1. Fidx
1. Level



In [2]:
url = 'https://raw.githubusercontent.com/lem0n4id/predicting-solar-flares-with-ML/main/data.csv'
data = pd.read_csv(url, header=0)
data = data.dropna()
print(data.shape)
print(list(data.columns))

(230, 8)
['Date', 'AR#', 'Location', 'Lgnl', 'Tflux', 'Ediss', 'Fidx', 'Level']


In [3]:
data.head()

,Date,AR#,Location,Lgnl,Tflux,Ediss,Fidx,Level
0,17/01/2005,10721,S04E03,0.0,0.0083,0.0073,0.01,0
1,23/01/2005,10726,N01W00,0.0,0.0731,0.0588,0.01,0
2,02/02/2005,10729,S10W09,0.0,0.0400,0.0366,1.63,0
3,08/02/2005,10731,S02W01,0.0,0.0084,0.0086,0.01,0
4,02/03/2005,10739,S03W03,0.0,0.0194,0.0162,0.84,0


In [4]:
cols=['Lgnl', 'Tflux', 'Ediss', 'Fidx']
X=data[cols]
y=data['Level']

In [5]:
X.head()

,Lgnl,Tflux,Ediss,Fidx
0,0.0,0.0083,0.0073,0.01
1,0.0,0.0731,0.0588,0.01
2,0.0,0.0400,0.0366,1.63
3,0.0,0.0084,0.0086,0.01
4,0.0,0.0194,0.0162,0.84


In [6]:
# spliting the data into training and test data

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)

In [7]:
# training the model

model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

In [8]:
# predicting the level of a solar flare - single entry

data_entry_row = 1
print(X_test.iloc[data_entry_row])
print()

level_p = model1.predict([X_test.iloc[data_entry_row]])

print("level of flare predicted: ", level_p[0])
print("actual level of flare: ", y_test.iloc[data_entry_row])


Lgnl      0.0893
Tflux     0.1074
Ediss     0.0994
Fidx     60.6000
Name: 90, dtype: float64

level of flare predicted:  2
actual level of flare:  1


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


## Visualizing results

Here, we're using a confusion matrix, accuracy score and a classification report to understand how our model is performing.

In [9]:
# transform confusion matrix into array

confmtrx = np.array(confusion_matrix(y_test, preds))


"""
Create DataFrame from confmtrx array 
rows for test: 0, 1, 2, 3
columns for preds: predicted_0, predicted_1, predicted_2, predicted_3
"""

pd.DataFrame(confmtrx, 
             index=['0', '1', '2', '3'], 
             columns=['predicted_0', 'predicted_1', 'predicted_2', 'predicted_3'])

,predicted_0,predicted_1,predicted_2,predicted_3
0,15,0,0,0
1,2,0,13,1
2,2,0,5,2
3,0,0,3,3


In [10]:
#Accuracy statistics
accuracy_score = metrics.accuracy_score(y_test, preds) * 100 # in percetage
print('Accuracy Score(in %):', accuracy_score)  

print()

#Create classification report
class_report=classification_report(y_test, preds, zero_division=0)
print(class_report)


Accuracy Score(in %): 50.0

              precision    recall  f1-score   support

           0       0.79      1.00      0.88        15
           1       0.00      0.00      0.00        16
           2       0.24      0.56      0.33         9
           3       0.50      0.50      0.50         6

    accuracy                           0.50        46
   macro avg       0.38      0.51      0.43        46
weighted avg       0.37      0.50      0.42        46



## Conclusion

- From the above cell, we can see that we get a accuracy score of 50% which is similar to what they got.

- We can also see the classification report which gives a good picture of how our model is performing.

- We observe that the model is performing really poor in distuinglishing level 2 solar flares. 

    This is probably due to the small size of data and lack of good enough parameters to enable the model to understand it.

---

The authors of the research paper mention that this method is promising, but needs a better set of features and a larger dataset.

Further research in this field is required.